# Particionado

## Creamos una sesión de spark

# Nueva sección

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 12.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 14.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=b1437e565a52a18a8ab576ec8286c4bde18de16cac42643d16ac74b7cabe0cf0
  Stored in directory: /home/datormx/.cache/pip/wheels/38/df/61/8c121f50c3cffd77f8178180dd232d90b3b99d1bd61fb6d6be
Successfully built pyspark


Spark permite desde la creación de la sesión o contexto, indicar la cantidad de particiones que tendremos

Para esto debemos de indicar con '[ ]'  en la indicación de master la cantidad total de particiones

In [2]:
spark = SparkSession.builder.appName("Particionado") \
.master( "local[5]").config("spark.driver.extraClassPath", "/content/sample_data/mysql-connector-j-8.3.0.jar").getOrCreate()

In [3]:
df = spark.range(0,20)
df.rdd.getNumPartitions()

5

El método 'parallelize', permite la asignar manualmente la cantidad de particiones.

In [29]:
rdd1 = spark.sparkContext.parallelize((0,20),6)
rdd1.getNumPartitions()

6

Del mismo modo cuandore creamos un RDD o DF, podemos hacer esto.

En el caso de los RDD se realiza de la siguiente forma



In [5]:
rddDesdeArchivo = spark \
    .sparkContext \
    .textFile("/content/sample_data/deporte.csv",10)

In [17]:
dfmysqlazure = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://mysqlservertestafabflexible.mysql.database.azure.com:3306/deportes") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "deporte") \
    .option("user", "mysqladmin") \
    .option("password", "Andres1980") \
    .load()

dfmysqlazure = dfmysqlazure.repartition(20)

# Mostrar los datos leídos
dfmysqlazure.show()


+----------+--------------------+
|deporte_id|      nombre_deporte|
+----------+--------------------+
|        34|      Figure Skating|
|        21|              Hockey|
|        15|       Alpine Skiing|
|        14|    Art Competitions|
|        22|              Rowing|
|        28|           Taekwondo|
|        35|                Golf|
|        44|    Freestyle Skiing|
|        66|         Aeronautics|
|        51|        Snowboarding|
|        31|            Canoeing|
|        46|        Trampolining|
|        39|Synchronized Swim...|
|        53|Short Track Speed...|
|        59|        Motorboating|
|        23|           Bobsleigh|
|        41|     Nordic Combined|
|        52|               Rugby|
|        30|              Diving|
|        24|             Fencing|
+----------+--------------------+
only showing top 20 rows



In [18]:
num_partitions = dfmysqlazure.rdd.getNumPartitions()
print("Número de particiones:", num_partitions)

Número de particiones: 20


In [7]:

rddDesdeArchivo.getNumPartitions()

Número de particiones: 10


10

Es una buena practica tener los archivos de datos particionados para una carga mas rápida y mejor administración.

El método 'saveAsTextFile' permite almacenar los archivos, particionados o no, en un ruta.

In [ ]:
rddDesdeArchivo.saveAsTextFile("/home/spark/Downloads/salidastexto/")

Convertir DataFrame a RDD y guardar como texto

In [22]:
dfmysqlazure.rdd.map(lambda row: ",".join([str(i) for i in row])).saveAsTextFile("/home/spark/Downloads/salidastexto2/")


In [ ]:
rddDesdeArchivo.saveAsTextFile("/home/spark/Downloads/salidastexto/")

In [20]:
!ls /home/spark/Downloads/salidastexto/

part-00000  part-00002	part-00004  part-00006	part-00008  _SUCCESS
part-00001  part-00003	part-00005  part-00007	part-00009


A continuación se muestra como cargar los multiples archivos en un mismo RDD.

Esta operación tambien se puede realizar para DF

In [24]:
rdd2 = spark.sparkContext.wholeTextFiles("/home/spark/Downloads/salidastexto2/*")

In [16]:
rdd = spark.sparkContext.wholeTextFiles("/home/spark/Downloads/salidastexto/*")

In [25]:
lista = rdd2.mapValues(lambda x : x.split()).collect()

In [17]:
lista = rdd.mapValues(lambda x : x.split()).collect()

In [26]:
l = [l[0] for l in lista]
l.sort()

In [27]:
rddDesdeArchivo = spark \
    .sparkContext \
    .textFile(','.join(l),
              10).map(lambda l : l.split(","))


In [28]:
rddDesdeArchivo.take(100)

[['34', 'Figure Skating'],
 ['21', 'Hockey'],
 ['15', 'Alpine Skiing'],
 ['14', 'Art Competitions'],
 ['22', 'Rowing'],
 ['28', 'Taekwondo'],
 ['35', 'Golf'],
 ['44', 'Freestyle Skiing'],
 ['66', 'Aeronautics'],
 ['51', 'Snowboarding'],
 ['31', 'Canoeing'],
 ['46', 'Trampolining'],
 ['39', 'Synchronized Swimming'],
 ['53', 'Short Track Speed Skating'],
 ['59', 'Motorboating'],
 ['23', 'Bobsleigh'],
 ['41', 'Nordic Combined'],
 ['52', 'Rugby'],
 ['30', 'Diving'],
 ['24', 'Fencing'],
 ['38', 'Volleyball'],
 ['3', 'Football'],
 ['62', 'Jeu De Paume'],
 ['4', 'Tug-Of-War'],
 ['58', 'Racquets'],
 ['17', 'Weightlifting'],
 ['37', 'Archery'],
 ['2', 'Judo'],
 ['27', 'Boxing'],
 ['5', 'Speed Skating'],
 ['42', 'Baseball'],
 ['48', 'Triathlon'],
 ['8', 'Ice Hockey'],
 ['60', 'Military Ski Patrol'],
 ['64', 'Alpinism'],
 ['33', 'Modern Pentathlon'],
 ['19', 'Luge'],
 ['18', 'Wrestling'],
 ['7', 'Athletics'],
 ['1', 'Basketball'],
 ['16', 'Handball'],
 ['57', 'Cricket'],
 ['61', 'Croquet'],
 ['56

In [ ]:
!pwd

/content


In [ ]:
spark